# Missed opportunity - 4th place solution

**Scores for version-11:-**

**public LB=0.9618**

**private LB=0.9482** 

# This is in reference to discussion here:
https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/169409

Yesterday, I saw a notebook claiming 0.9565 score in the notebook title but it was not submitted. Thus, it was not showing in high scoring notebooks. I opened the notebook and saw he has ensembled gdbt predictions with EFNs. The gdbt alone was giving around 0.7 score. So, I submitted the predictions to see if it actually works. The kernel was made private just after 2-3 hours. So, some teams (that includes me) took advantage of it to climb public lb. Although, the notebook may be overfitting like many high scoring kernels, I feel the guilt of private sharing. So, I am making this kernel public to make things rights !! The dataset was kept private in the actual notebook.

# Beware
This notebook along with many high scoring notebook may be just overfitting to public leaderboard. So, Making submissions with this may not work well with private data. There has been pretty drastic shakeups in many competitions recently. And at least in this competition, there is a high scope of overfitting to public LB. So, here is a small advice, whatever final submission you choose, pick a good CV technique and trust your CV score. 

The main idea is to train an XG Boost model on tabular data and perform voting ensemble with efn predictions. Although, XG Boost alone scores 0.6985, when ensembled with efn predictions (0.9545), it gives a boost of 0.002 score to 0.9565. But as I said, validate this with your CV.

# Scores:
1. Without gradient boosting model: 0.9545
2. Only gradient boosting model: 0.6985
3. Ensemble: 0.9565

I don't remember the name of kernel author so I can't give references here.

In [ ]:
!pip install xgboost

import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [ ]:
train= pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test= pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
sub= pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')

In [ ]:
train['sex'] = train['sex'].fillna('na')
train['age_approx'] = train['age_approx'].fillna(0)
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].fillna('na')

test['sex'] = test['sex'].fillna('na')
test['age_approx'] = test['age_approx'].fillna(0)
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].fillna('na')
train['sex'] = train['sex'].astype("category").cat.codes +1
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].astype("category").cat.codes +1
train.head()

In [ ]:
test['sex'] = test['sex'].astype("category").cat.codes +1
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].astype("category").cat.codes +1
test.head()

In [ ]:
x_train = train[['sex', 'age_approx','anatom_site_general_challenge']]
y_train = train['target']


x_test = test[['sex', 'age_approx','anatom_site_general_challenge']]
# y_train = test['target']


train_DMatrix = xgb.DMatrix(x_train, label= y_train)
test_DMatrix = xgb.DMatrix(x_test)

In [ ]:
clf = xgb.XGBClassifier(n_estimators=2600, 
                        max_depth=15, 
                        objective='multi:softprob',
                        seed=0,  
                        nthread=-1, 
                        learning_rate=0.15, 
                        num_class = 2, 
                        scale_pos_weight = (32542/584))

clf.fit(x_train, y_train)

In [ ]:
clf.predict_proba(x_test)[:,1]
# clf.predict(x_test)
sub.target = clf.predict_proba(x_test)[:,1]
sub_tabular = sub.copy()

In [ ]:
sub_tabular.to_csv('submission_tabular.csv', index = False)

In [ ]:
# Ensemble 
sub_new = pd.read_csv("../input/siimisic-melanoma-subs/submission_b_Fork-of-ensemble_melanoma-ac9964_AjayKumar_v12_LB0p9596.csv")

#sub_mean = pd.read_csv("../input/siimisic-melanoma-subs/Submissions_MeanOf_26Models_LB0pt9569.csv") # LB=0.9569
sub_mean = pd.read_csv("../input/siimisic-melanoma-subs/Submissions_MeanOf_27Models_LB0pt9571.csv") # LB=0.9571

sub_tuned = pd.read_csv("../input/simple-public-blender-0-930/submission.csv") # ver-86/88 LB=0.9528


sub.target = sub_mean.target *0.2 + sub_new.target *0.5 + sub_tabular.target *0.2 + sub_tuned.target *0.1


In [ ]:
sub.to_csv('ensemble_submission.csv', index = False)
sub.head(20)